# MobileNetV2 Leaf Disease Classifier

This notebook is for training/fine-tuning a MobileNetV2 model for crop/leaf disease detection.

- Place your dataset (images + labels) in the `notebooks/` directory.
- Dataset should be in a folder format (e.g., `train/class_name/*.jpg`).
- Outputs a PyTorch `.pth` file and TorchServe `.mar` archive.
    

In [6]:
import torch
import torchvision
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import os
from tqdm import tqdm  # for progress bar

# Paths
data_dir = '../data/PlantVillage'
num_classes = len(os.listdir(os.path.join(data_dir, 'train')))

# Transforms
train_transforms = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

train_dataset = datasets.ImageFolder(os.path.join(data_dir, 'train'), train_transforms)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Model
model = models.mobilenet_v2(weights='IMAGENET1K_V1')
model.classifier[1] = torch.nn.Linear(model.last_channel, num_classes)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# Loss and Optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

best_accuracy = 0.0

# Training Loop with progress bar and accuracy
for epoch in range(5):  # reduce to 5 epochs for faster debug
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/5")
    for images, labels in loop:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        loop.set_postfix(loss=loss.item(), acc=100. * correct / total)

    epoch_loss = running_loss / len(train_loader)
    epoch_acc = 100. * correct / total
    print(f"\n✅ Epoch {epoch+1} complete. Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%")

    # Save best model
    if epoch_acc > best_accuracy:
        best_accuracy = epoch_acc
        torch.save(model.state_dict(), 'mobilenetv2_leaf_disease_best.pth')
        print("💾 Saved best model so far!")

# Save final model
torch.save(model.state_dict(), 'mobilenetv2_leaf_disease_final.pth')
print("🎉 Final model saved.")


Epoch 1/5: 100%|██████████| 516/516 [18:20<00:00,  2.13s/it, acc=92.6, loss=0.037] 



✅ Epoch 1 complete. Loss: 0.2932, Accuracy: 92.61%
💾 Saved best model so far!


Epoch 2/5: 100%|██████████| 516/516 [14:34<00:00,  1.70s/it, acc=98.8, loss=0.119]  



✅ Epoch 2 complete. Loss: 0.0461, Accuracy: 98.79%
💾 Saved best model so far!


Epoch 3/5: 100%|██████████| 516/516 [17:17<00:00,  2.01s/it, acc=99.2, loss=0.00533]



✅ Epoch 3 complete. Loss: 0.0276, Accuracy: 99.19%
💾 Saved best model so far!


Epoch 4/5: 100%|██████████| 516/516 [32:30<00:00,  3.78s/it, acc=99.3, loss=0.00584]   



✅ Epoch 4 complete. Loss: 0.0226, Accuracy: 99.34%
💾 Saved best model so far!


Epoch 5/5: 100%|██████████| 516/516 [21:28<00:00,  2.50s/it, acc=99.5, loss=0.0104]   


✅ Epoch 5 complete. Loss: 0.0179, Accuracy: 99.48%
💾 Saved best model so far!
🎉 Final model saved.


## TorchServe Model Packaging

After training, use the following command to create a TorchServe .mar file:

```bash
torch-model-archiver --model-name leaf-disease --version 1.0 --serialized-file mobilenetv2_leaf_disease.pth --handler image_classifier --extra-files index_to_name.json --export-path model_store
```

- `index_to_name.json` should map class indices to disease names.
    